I dati utilizzati in questo notebook sono stati presi dalla competizione di Analytics Vidhya [Practice Problem: Big Mart Sales III](https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/#data_dictionary).

### Riferimenti bibliografici:

* Azzalini, A. &  Scarpa B. (2012), [Data Analysis and Data Mining: An Introduction](https://global.oup.com/academic/product/data-analysis-and-data-mining-9780199767106?q=Data%20Mining&lang=en&cc=it).
* Hastie, T.; Tibshirani, R. & Friedman, J. (2009), [The Elements of Statistical Learning](https://web.stanford.edu/~hastie/ElemStatLearn/).

# Regressione stepwise, Ridge e LASSO

## Indice

1. [Alcune definizioni utili](#definizioni)<br />
2. [Decidere la metrica di valutazione](#metrica)<br />
3. [Creare una baseline](#baseline)<br />
    3.1 [`DummyRegressor`](#dummy_regressor)<br />
    3.2 [Regressione lineare](#regressione_lineare)<br />
4. [Regressione stepwise](#stepwise)<br />
5. [Ridge](#ridge)<br />
    5.1 [Selezione dell'iperparametero `alpha`](#alpha_ridge)<br />
    5.2 [Risultati dell'esperimento](#risultati_ridge)<br />
6. [LASSO (Least Absolute Shrinkage and Selection Operator)](#lasso)<br />
    6.1 [Selezione dell'iperparametero `alpha`](#alpha_lasso)<br />
    6.2 [Risultati dell'esperimento](#risultati_lasso)<br />
7. [Valutare le performance sull'insieme di test](#performance_test)<br />
8. [Utilizzare il modello su dati nuovi](#dati_nuovi)<br />
    8.1 [Utilizzare la pipeline di preprocessamento e lo stimatore già allenati](#allenati)<br />
    8.2 [Riallenare la pipeline da zero e prevedere le vendite per i nuovi dati](#riallenare)<br />

In [ ]:
import inspect
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

> Nota: per la descrizione del problema e dei dati vedere il notebook 07_analisi_esplorativa_e_preprocessamento_dei_dati.ipynb.

In [ ]:
PATH = "output/07/"

X_train = pd.read_pickle(PATH + "/X_train.pkl")
X_val = pd.read_pickle(PATH + "/X_val.pkl")
X_test = pd.read_pickle(PATH + "/X_test.pkl")
y_train = pd.read_pickle(PATH + "/y_train.pkl")
y_val = pd.read_pickle(PATH + "/y_val.pkl")
y_test = pd.read_pickle(PATH + "/y_test.pkl")

# 1. Alcune definizioni utili <a id=definizioni> </a>

Sia $y_i$ l'osservazione i-esima della variabile risposta, $\overline{y}$ la media degli $y_i$ e $\hat{y}_i$ la stima di $y_i$ data dal modello, si definiscono le seguenti quantità:

**Somma dei quadrati dei residui**:
$$
\mathrm{RSS} = \sum\limits_{i=1}^n(y_i - \hat{y}_i)^2
$$

**Somma dei quadrati totale**:
$$
\mathrm{TSS} = \sum\limits_{i=1}^n(y_i - \overline{y}_i)^2
$$

**Coefficiente di determinazione**:
$$
R^2 = 1 - \frac{\mathrm{RSS}}{\mathrm{TSS}}
$$

**Errore quadratico medio / stima della varianza dei residui**:
$$
\mathrm{MSE} = \hat{\sigma}^2 = \frac{\mathrm{RSS}}{n}
$$

**Radice dell'errore quadratico medio**:
$$
\mathrm{RMSE} = \sqrt{\mathrm{MSE}}
$$

**Criterio d'informazione di Akaike**:
$$
\mathrm{AIC} = 2k - 2\ln(\hat{L})
$$

**Vaolore massimo della log-verosimiglianza, caso errori i.i.d. $\sim{\mathcal{N}}(0,\sigma^2)$**:
$$
\ln{(\hat{L})} = -\frac{n}{2}\ln(2\pi) - \frac{n}{2}\ln({\hat{\sigma}}^2) - \frac{1}{2{\hat{\sigma}}^2}\mathrm{RSS}
$$

# 2. Decidere la metrica di valutazione <a id=metrica> </a>

### Evaluation Metric
​
Your model performance will be evaluated on the basis of your prediction of the sales for the test data (test.csv), which contains similar data-points as train except for the sales to be predicted. Your submission needs to be in the format as shown in "SampleSubmission.csv".
​
We at our end, have the actual sales for the test dataset, against which your predictions will be evaluated. We will use the Root Mean Square Error value to judge your response.
​
$
RMSE = \sqrt{\frac{\sum_{i=1}^N(Predicted_i - Actual_i)^2}{N}}
$
​
Where,
$N$: total number of observations
Predicted: the response entered by user
Actual: actual values of sales
​
Also, note that the test data is further divided into Public (25%) and Private (75%) data. Your initial responses will be checked and scored on the Public data. But, the final rankings will be based on score on Private data set. Since this is a practice problem, we will keep declare winners after specific time intervals and refresh the competition.

In [ ]:
from msbd.metriche import radice_errore_quadratico_medio

print(inspect.getsource(radice_errore_quadratico_medio))

# 3. Creare una baseline <a id=baseline> </a>

## 3.1 `DummyRegressor` <a id=dummy_regressor> </a>

In [ ]:
from sklearn.dummy import DummyRegressor

In [ ]:
dr = DummyRegressor(strategy='mean')

dr.fit(X_train, y_train)

In [ ]:
print("R2 training: {:.4f}".format(dr.score(X_train, y_train)))
print("R2 validation: {:.4f}".format(dr.score(X_val, y_val)))
print("RMSE training: {:.4f}".format(radice_errore_quadratico_medio(y_train, dr.predict(X_train))))
print("RMSE validation: {:.4f}".format(radice_errore_quadratico_medio(y_val, dr.predict(X_val))))

### Esercizio

1. È possibile ottenere un [coefficiente di determinazione](https://it.wikipedia.org/wiki/Coefficiente_di_determinazione) $R^2$ negativo?
2. Ci stupisce che l'$R^2$ sull'insieme di training sia esattamente zero per il DummyRegressor?
3. Se si valuta uno stimatore sull'insieme di validation, è necessario utilizzare l'$\bar{R}^2$ ($R^2$ corretto) al posto del $R^2$? 

Motivare le risposte.

## 3.2 Regressione lineare <a id=regressione_lineare> </a>

> Nota: per un'implementazione del modello lineare e un summary più vicini a quelli di R considerare la classe [`OLS()`](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html) di [`statsmodels`](https://www.statsmodels.org/stable/index.html).

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

lr.fit(X_train, y_train)

In [ ]:
from msbd.grafici import grafico_coefficienti

print(inspect.getsource(grafico_coefficienti))

In [ ]:
plt.figure(figsize=(15, 3))

print("Intercetta: {:.2f}".format(lr.intercept_))
grafico_coefficienti(lr.coef_, X_train.columns)

plt.show()

print("R2 training: {:.4f}".format(lr.score(X_train, y_train)))
print("R2 validation: {:.4f}".format(lr.score(X_val, y_val)))
print("RMSE training: {:.4f}".format(radice_errore_quadratico_medio(y_train, lr.predict(X_train))))
print("RMSE validation: {:.4f}".format(radice_errore_quadratico_medio(y_val, lr.predict(X_val))))

# 4. Regressione stepwise <a id=stepwise> </a>

In [ ]:
from msbd.selezione_variabili import Stepwise

print(inspect.getsource(Stepwise))

In [ ]:
from msbd.metriche import criterio_informazione_akaike

print(inspect.getsource(criterio_informazione_akaike))

In [ ]:
stepwise = Stepwise(LinearRegression(), criterio_informazione_akaike, "avanti", verboso=True)

stepwise.fit(X_train, y_train)

### Esercizio

L'implementazione del metodo `_selezione_avanti()` della classe `Stepwise` è generico ma computazionalmente oneroso, ad ogni chiamata di `_passo_avanti()` viene riallenato il modello una volta per ogni variabile considerata in quel passo.

1. Sia $n=49$ il numero di variabili a disposizione (come nel nostro caso). Al massimo quante volte viene allenato il modello?
2. Sia sempre $n=49$. Quanti sono tutti i possibili sottoinsiemi di variabili?

In [ ]:
plt.figure(figsize=(15, 3))

print("Intercetta: {:.2f}".format(stepwise.stimatore_.intercept_))
grafico_coefficienti(
    stepwise.stimatore_.coef_,
    stepwise.variabili_selezionate_
)

plt.show()

print("R2 training: {:.4f}".format(stepwise.score(X_train, y_train)))
print("R2 validation: {:.4f}".format(stepwise.score(X_val, y_val)))
print("RMSE training: {:.4f}".format(radice_errore_quadratico_medio(y_train, stepwise.predict(X_train))))
print("RMSE validation: {:.4f}".format(radice_errore_quadratico_medio(y_val, stepwise.predict(X_val))))

### Esercizio

1. Completare i metodi `_passo_indietro()`, `_selezione_avanti()`, `_selezione_ibrida()`;
2. Ripetere l'esercizio con `procedura="indietro"`. Si ottiene lo stesso insieme di variabili che con `procedura="avanti"`?
3. Ripetere l'esercizio con `procedura="ibrida"`. Si ottiene lo stesso insieme di variabili che con `procedura="avanti"`?
4. Nel caso della regressione lineare, come si potrebbe velocizzare la selezione quando `procedura="indietro"`?

# 5. Ridge <a id=ridge> </a>

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
from msbd.esperimenti import esperimento_regolarizzazione

print(inspect.getsource(esperimento_regolarizzazione))

## 5.1 Selezione dell'iperparametero `alpha` <a id=alpha_ridge> </a>

In [ ]:
alpha_list = [1e-2, 1e-1, 1e-0, 1e+1, 1e+2, 1e+3, 1e+4]

esperimento = esperimento_regolarizzazione(Ridge(), alpha_list, X_train, y_train, X_val, y_val)

## 5.2 Risultati dell'esperimento <a id=risultati_ridge> </a>

### $R^2$ e $\mathrm{RMSE}$

In [ ]:
esperimento[["R2_Train", "R2_Val", "RMSE_Train", "RMSE_Val"]]

### Coefficienti stimati al variare di `alpha`

In [ ]:
ax = esperimento[X_train.columns].plot(figsize=(15, 5), legend=None, title="Grafico dei coefficienti Ridge al variare di alpha", use_index=False, lw=2)
plt.xticks(range(len(alpha_list)), alpha_list)
plt.legend(loc='center', bbox_to_anchor=(0.5, -0.5), ncol=4)
plt.xlabel("Alpha")

plt.show()

# 6. LASSO (Least Absolute Shrinkage and Selection Operator) <a id=lasso> </a>

In [ ]:
from sklearn.linear_model import Lasso

## 6.1 Selezione dell'iperparametero `alpha` <a id=alpha_lasso> </a>

In [ ]:
alpha_list = [1e-3, 1e-2, 1e-1, 1e-0, 1e+1, 1e+2, 1e+3]

esperimento = esperimento_regolarizzazione(Lasso(), alpha_list, X_train, y_train, X_val, y_val)

## 6.2 Risultati dell'esperimento <a id=risultati_lasso> </a>

### $R^2$ e $\mathrm{RMSE}$

In [ ]:
esperimento[["R2_Train", "R2_Val", "RMSE_Train", "RMSE_Val"]]

### Coefficienti stimati al variare di `alpha`

In [ ]:
ax = esperimento[X_train.columns].plot(figsize=(15, 5), legend=None, title="Grafico dei coefficienti Lasso al variare di alpha", use_index=False, lw=2)
plt.xticks(range(len(alpha_list)), alpha_list)
plt.legend(loc='center', bbox_to_anchor=(0.5, -0.5), ncol=4)
plt.xlabel("Alpha")

plt.show()

# 7. Valutare le performance sull'insieme di test <a id=performance_test> </a>

### Esercizio

Scegliere lo stimatore migliore (ed eventuali iperparametri) tra quelli analizzati.

In [ ]:
# TODO: sostituire DummyRegressor con lo stimatore scelto
# ============== YOUR CODE HERE ==============
stimatore = DummyRegressor()
# ============== YOUR CODE HERE ==============

In [ ]:
X_trainval = X_train.append(X_val)
y_trainval = y_train.append(y_val)

stimatore.fit(X_trainval, y_trainval)

In [ ]:
print("R2 training + validation: {:.4f}".format(stimatore.score(X_trainval, y_trainval)))
print("R2 test: {:.4f}".format(stimatore.score(X_test, y_test)))
print("RMSE training + validation: {:.4f}".format(radice_errore_quadratico_medio(y_trainval, stimatore.predict(X_trainval))))
print("RMSE test: {:.4f}".format(radice_errore_quadratico_medio(y_test, stimatore.predict(X_test))))

# 8. Utilizzare il modello su dati nuovi <a id=dati_nuovi> </a>

### Leggere i dati

In [ ]:
PATH = "datasets/big_mart_sales"

X_new = pd.read_csv(PATH + "/Test_u94Q5KV.csv")
print("Dimensione del dataset: {} x {}".format(*X_new.shape))
X_new.head()

## 8.1 Utilizzare la pipeline di preprocessamento e lo stimatore già allenati <a id=allenati> </a>

In [ ]:
from sklearn.externals import joblib

In [ ]:
PATH = "output/07"

pipeline = joblib.load(PATH + "/preproc.joblib")

In [ ]:
pipeline.transform(X_new).head() # non stiamo salvando X_new dopo la trasformazione, serve solo come esempio

In [ ]:
pipeline.steps.append(("stimatore", stimatore)) # appendo lo stimatore alla pipeline di preprocessamento
pipeline.named_steps.keys()

> Nota: `pipeline.predict()` applica in sequenza gli step di preprocessamento definiti nella pipeline preproc.joblib a `X_new` e sucessivamente applica il metodo `predict()` dello stimatore allenato sopra.

In [ ]:
y_pred = pipeline.predict(X_new)
y_pred

## 8.2 Riallenare la pipeline da zero e prevedere le vendite per i nuovi dati <a id=riallenare> </a>

In [ ]:
PATH = "datasets/big_mart_sales" # cambiare in base a dove si è salvato il dataset

dati = pd.read_csv(PATH + "/Train_UWu5bXk.csv")
print("Dimensione del dataset: {} x {}".format(*dati.shape))
display(dati.head())

risposta = "Item_Outlet_Sales"
esplicative = sorted(col for col in dati.columns if col != risposta)

X, y = dati[esplicative].copy(), dati[risposta].copy()

In [ ]:
pipeline.fit(X, y) # riallenamento della pipeline

In [ ]:
y_pred = pipeline.predict(X_new)
y_pred

### Esercizio

1. Aggiungere a X_new la colonna *Item_Outlet_Sales_Pred*;
2. Ordinare il X_new in base a *Item_Outlet_Sales_Pred* in ordine discendente;
3. Commentare le prime e le ultime 5 righe.